# Homework 2 - TF-IDF Classifier

Ваша цель обучить классификатор который будет находить "токсичные" комментарии и опубликовать решения на Kaggle [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

В процессе обучения нужно ответить на ***[вопросы](https://docs.google.com/forms/d/e/1FAIpQLSd9mQx8EFpSH6FhCy1M_FmISzy3lhgyyqV3TN0pmtop7slmTA/viewform?usp=sf_link)***

Данные можно скачать тут - https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data



In [1]:
import numpy as np
import pandas as pd
import os
import collections

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
path_dir = os.path.join('D:','DatasetsAI','toxic_comments')
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv(os.path.join(path_dir,'train.csv')).fillna(' ')
test = pd.read_csv(os.path.join(path_dir,'test.csv')).fillna(' ')

Стадартными подходами для анализа текста являются [Bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) и его модификация [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Они реалзованны в `sklearn` в виде [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

Более подробней про них можно посмотреть [тут](https://github.com/udsclub/workshop/blob/master/notebooks/UDS-workshop-feature-extraction-and-engineering.ipynb)

In [3]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [4]:
import re
def get_counter_freqs(df):
    all_text = "".join(df.tolist())
    #delim by all punctuation except for apostrophe
    all_text = re.findall(r"[^\W]+'*", all_text.lower())
    return collections.Counter(all_text)

In [5]:
word_counter = get_counter_freqs(all_text)
print(word_counter.most_common(1))

[('the', 917477)]


In [7]:
# Попробуйте разные Vectorizer и разные размеры n-gramm, стоп-слова, обрезку редких слов, обрезку слишком частых слов
def get_vectorizer(count_vectorizer=True):
    if count_vectorizer:
        return CountVectorizer(stop_words='english', min_df=2, ngram_range=(1, 4), analyzer='char', sublinear_tf=True,
                               token_pattern=r"[^\W]+'*")
    return TfidfVectorizer(stop_words='english', min_df=2, ngram_range=(1, 4), analyzer='char', sublinear_tf=True,
                           token_pattern=r"[^\W]+'*")

In [8]:
word_vectorizer = get_vectorizer(False)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

Для классификации будем использовать логистическую регрессию [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [9]:
classifier = LogisticRegression(C=1.35, class_weight='balanced')

Будем тренировать по одному классификатору на каждый класс. 

Что бы провалидировать качество модели воспользуемся функцией [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

In [10]:
scores= []

for class_name in class_names:
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, scoring='roc_auc', n_jobs=2))
    
    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)

print('Total score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9754791104909639
CV score for class severe_toxic is 0.9891553922853089
CV score for class obscene is 0.9892621105314424
CV score for class threat is 0.9879265867227218
CV score for class insult is 0.9819600695922416
CV score for class identity_hate is 0.9830072261402084
Total score is 0.9844650826271478


Попробуйте подобрать лучшие параметры для `word_vectorizer` и `classifier` оптимизируя метрику [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)


---

Опубликуйте лучшие решение на [Kaggle Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/submit)

In [11]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [13]:
for class_name in class_names:
    train_target = train[class_name]
    classifier.fit(train_word_features, train_target)
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]    

In [ ]:
submission.to_csv('submission.csv', index=False)